In [1]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)


def modify_paths(config):
    # dataset.data_dir に ../ を追加
    config['dataset']['data_dir'] = os.path.join('..', config['dataset']['data_dir'])

    # dataset.split_config が存在する場合に ../ を追加
    if 'split_config' in config['dataset']:
        config['dataset']['split_config'] = os.path.join('..', config['dataset']['split_config'])

    return config


In [ ]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/dataset/gen1/event_frame/sequence/dt_50.yaml',
        '../config/experiment/sequence/train.yaml'
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
config = modify_paths(merged_conf)
dynamically_modify_train_config(config=config)

In [ ]:
from modules.fetch import fetch_model_module, fetch_data_module
data = fetch_data_module(config=config)
model = fetch_model_module(config=config)
data.setup('fit')
model.setup('fit')

In [4]:
train_dataset = data.train_dataset

In [ ]:
from utils.visualization import plot_images_yolox
index = 100
sample =  train_dataset[index]
events = sample['events']
labels = sample['labels']
plot_images_yolox(events, labels, mode='train', subplot=True, cols=3)

In [6]:
import torch
input = torch.randn(1, 3, 320, 320).to(torch.float32)
prev_states = None

In [ ]:
model.eval()
out, state = model.model(input, prev_states)